# Acceleration to Velocity conversion: Waveform

### Acceleration waveform to Velocity waveform conversion using the composite trapezoidal rule

> Remark: 
* With the trapezoidal rule, we approximate the curve by using piecewise linear functions. 
* With Simpson's rule, we approximate the curve by using piecewise quadratic functions.
* `Here we have used trapezoidal rule for obtaining velocity waveform by numerical integration over acceleration.`
* `This can be improved further using Simpson's rule in future.`


***    
   
<br>     
  
***Matlab code for acceleration to velocity waveform conversion- numerical integration.***  
*`Shared by Vijay`*  
***

```python
fs=48828.125; %Sampling frequency
VibAccelerationValues=Tbl.metric_value; %Read raw vibration acceleration signal
VibAccelerationValues=VibAccelerationValues-mean(VibAccelerationValues); %Substract mean value
Time=0:1/fs:(length(VibAccelerationValues)-1)/fs; %Define time vector
Time=Time';
                  
VibVelocityValues=cumtrapz(VibAccelerationValues,Time); %Integrate acceleration signal to obtain velocity signal
VibVelocityValues=VibVelocityValues-mean(VibVelocityValues); %Subtract mean value
```   

***

***  
***  

**Import requied libraries**

In [ ]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
from plotly.offline import init_notebook_mode
import plotly.io as pio
import plotly.graph_objects as go
init_notebook_mode(connected = True)
pio.templates.default = "simple_white"
import matplotlib.pyplot as plt

from scipy import integrate

pd.set_option('display.max_rows', 5000000)
pd.set_option('display.max_columns', 5000000)
pd.set_option('display.width', 100000000)

def accln_to_velocity_v2(amp, freq, G=1):
    """
    freq = Frequency in Hz.
    amp = amplitude in acceleration spectrum.
         (If we convert from RMS then the resulting velocity amplitude will be in RMS.)
    G = G in these formulas is not the acceleration of gravity. 
        It is a constant for calculation within different systems. 
        For metric, G is 9.80665 m/s². For Imperial, G is 386.0885827 in/s² For SI, G is 1 m/s²
    
    * In below formula, multilying nominator with 1000 to convert acceleration amplitude from m/s2 to mm/s2 
    """
    try:
        amp_velocity = (G * amp * 1000)/(2 * np.pi * freq)
        return(amp_velocity)
    except ZeroDivisionError:
        return(0)
    
    

def get_spec_from_waveform(df_wf, y_var_name = "amp_acceleration", plot = False):
    """
    get spectrum data from waveform, df_wf should have x(time) and y(amplitude) columns
    """
    N = df_wf.shape[0]; # N # Number of Sample Points
    spacing = df_wf['x'][N-1]/N
    Y    = np.fft.fft(df_wf['y'])
    freq = np.fft.fftfreq(N, spacing)
    df_spectrum = pd.DataFrame( freq, Y)
    #d = {'col1': [1, 2], 'col2': [3, 4]}
    d = {'freq':  freq[:N//2], y_var_name: 2.0/N * np.abs(Y[:N//2])}
    #pd.DataFrame( freq[:N//2], 2.0/N * np.abs(Y[:N//2]))
    #df_filtered_spectrum = pd.DataFrame( freq[:N//2], 2.0/N * np.abs(Y[:N//2]))
    df_filtered_spectrum = pd.DataFrame( data = d)
    if plot:
        fig, ax = plt.subplots()
        ax.plot(freq[:N//2], 2.0/N * np.abs(Y[:N//2]))
        plt.show()
    return(df_filtered_spectrum, df_spectrum)

### Load waveform data - csv files

In [ ]:
# Read waveform data sample
wav_df1 = pd.read_csv('../waveform-fft/g-pump-5_pump-axial_raw-data_2020-03-05 09_15_58_1480_td.csv')
wav_df2 = pd.read_csv('../waveform-fft/g-pump-5_pump-axial_raw-data_2020-03-05 10_15_59_1480_td.csv')

In [ ]:
wav_df1.head(2)

In [ ]:
print('Data collection duration (in sec): ', max(wav_df1.x))
print('Number of Data Points collected: ', wav_df1.shape[0])
print('Sampling Frequency of the wavform: ',wav_df1.shape[0]/max(wav_df1.x))

In [ ]:
# print(np.arange(0.0, max(wav_df1.x), 1/(wav_df1.shape[0]/max(wav_df1.x))))
# print(wav_df1['x'].values)

In [ ]:
fs=48828.125; #Sampling frequency of the waveform   # wav_df1.shape[0]/max(wav_df1.x)
VibAccelerationValues = wav_df1['y'].values; #Read raw vibration acceleration signal
Time= wav_df1['x'].values #np.arange(0.0, max(wav_df1.x), 1/(wav_df1.shape[0]/max(wav_df1.x))) #Define time vector

## Remove offset of given signals - An approximate. This approach can be improved later on. 
## mean calculates the given offset
## acceleration value has to be zero without offset!
VibAccelerationValues = VibAccelerationValues - np.mean(VibAccelerationValues)  #Substract mean value
VibVelocityValues = integrate.cumtrapz(x = Time, y = VibAccelerationValues, initial=0) #Integrate acceleration signal to obtain velocity signal
VibVelocityValues = VibVelocityValues-np.mean(VibVelocityValues)                #Subtract mean value

In [ ]:
print('Length of obtained velocity waveform: ',len(VibVelocityValues))
print('\nMean of acceleration waveform (m/s2): ', np.mean(VibAccelerationValues))
print('Mean of velocity waveform (mm/s): ', 1000*np.mean(VibVelocityValues))
print('\nRMS of acceleration waveform (m/s2): ', np.sqrt(np.mean(VibAccelerationValues**2)))
print('RMS of velocity waveform (mm/s): ', 1000*np.sqrt(np.mean(VibVelocityValues**2)))

#### Top few rows from Acceleration Waveform

In [ ]:
wav_df1.head(10)

#### Top few rows from velocity waveform

In [ ]:
df_1 = pd.DataFrame(data= {'x': list(Time), 'y': list(VibVelocityValues)})
df_1.head(10)

## Visualize Wavforms:

### Acceleration Wavform
**Amplitude in m/s2**

In [ ]:
fig = px.line(wav_df1, x='x', y='y', template='plotly_white', width=None, height=None, title = 'Acceleration Wavform')
fig.show()

### Velocity Wavform
**Amplitude in mm/s**

In [ ]:
fig = px.line(df_1, x='x', y=df_1['y']*1000, template='plotly_white', width=None, height=None, title = 'Velocity Wavform')
fig.show()

## Waveform to Spectrum conversion: FFT


### Acceleration Waveform to Acceleration Spectrum

In [ ]:
fft1, _ = get_spec_from_waveform(wav_df1, 'y', plot = False)
fig = px.line(fft1, x='freq', y='y', template='plotly_white', width=None, height=None)
fig.update_xaxes(range=(0,2000))
fig.show()

### Acceleration Spectrum to Velocity Spectrum
#### Formula applied on acceleration spectrum

In [ ]:
fft1['y2'] = list(map(accln_to_velocity_v2, fft1['y'], fft1['freq'])); #df_spec.head()
fig = px.line(fft1, x='freq', y='y2', template='plotly_white', width=None, height=None)
fig.update_xaxes(range=(0,2000))
fig.show()

### Velocity Waveform to Velocity Spectrum 
#### FFT on Velocity wavform

In [ ]:
fft1_1, _ = get_spec_from_waveform(df_1, 'y', plot = False)
fig = px.line(fft1_1, x='freq', y=fft1_1['y']*1000, template='plotly_white', width=None, height=None)
fig.update_xaxes(range=(0,2000))
fig.show()

## Compairing both Method of Velocity Spectrum Conversion
#### Method-1:
**First convert acceleration waveform to velocity waveform and apply FFT on that to get velocity spectrum.**

#### Method-2:
**First apply FFT on acceleration waveform to get acceleration spectrum and apply formula on that to get velocity spectrum.**

In [ ]:
fft1_1, _ = get_spec_from_waveform(df_1, 'y', plot = False)
fft1_2, _ = get_spec_from_waveform(wav_df1, 'y', plot = False)
fft1_2['y2'] = list(map(accln_to_velocity_v2, fft1_2['y'], fft1_2['freq'])); #df_spec.head()

fig = go.Figure() #Add traces into it
fig.add_trace(go.Scatter(x=fft1_1['freq'], y=fft1_1["y"]*1000, mode='lines', name="Method-1"))
fig.add_trace(go.Scatter(x=fft1_2['freq'], y=fft1_2["y2"], mode='lines', name="Method-2"))
fig.update_xaxes(range=(0,2000))
fig.show()

In [ ]:
fft1_1['y']  = fft1_1['y']*1000
fft1_1.head()

In [ ]:
fft1_2[['freq', 'y2']].head()

***

### Conclusion: 

#### 1. To obtain velocity waveform use integration over acceleration waveform.
#### 2. FFT on acceleration waveform to obtain acceleration spectrum.
#### 3. Apply unit conversion formula on acceleration spectrum to obtain velocity spectrum.


***